# Récupération et traitement des données

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import contextily as ctx
import pynsee as yns
import requests
from cartiflette.s3 import download_vectorfile_url_all

/home/cathu/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from pathlib import Path

In [3]:
Path.cwd()

PosixPath('/home/cathu/Documents/ENSAE/Projet_Catherine_Christelle')

## Les aménagements cyclables en Ile de France

In [4]:
a_velo= gpd.read_file('amenagements-velo-en-ile-de-france.geojson')

In [5]:
# Nombre d'entrées
a_velo.shape[0]

112217

In [6]:
a_velo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Un préalable au calcul des surfaces est le choix du système de projection adéquat. Dans notre cas , il s'agit de convertir les données au système de projection Lambert 93 qui est le plus approprié.

In [7]:
a_velo['longueur'].describe()

count    112217.000000
mean        126.315959
std         174.519817
min           0.000000
25%          24.000000
50%          70.000000
75%         161.000000
max        3984.000000
Name: longueur, dtype: float64

In [8]:
a_velo.sample(n=1)

,osm_id,nom_com,sens_voit,ag,panneaux,moyenn_ech,revetement,highway,insee_com,nom_voie,longueur,petite_ech,nv,ad,geometry
95503,71122991.0,1er Arrondissement,DOUBLE,None,None,24,asphalt,tertiary,75101,Rue Étienne Marcel,92,2,limite 30,voie bus uni,"LINESTRING (2.34511 48.86480, 2.34530 48.86476..."


Pour le moment, nous travaillons sans les données de géolocalisation qui ne sont pas nécessaires aux calculs. Nous créons pour cela une nouvelle dataframe

In [9]:
col_to_keep = ["nom_com", "sens_voit", "ag", "panneaux", "revetement", "highway", "insee_com", "longueur", "nv", "ad"] 

In [10]:
# df_a_velo = a_velo[col_to_keep].copy()

df_a_velo = a_velo

print(df_a_velo.head())

         osm_id            nom_com sens_voit                              ag  \
0  4.014620e+08            Chelles    DOUBLE                            None   
1  4.048666e+08   La Queue-en-Brie        NC  chemin service site propre uni   
2  1.154304e+09            Lésigny        NC                  voie verte uni   
3  3.300503e+08  Pontault-Combault    UNIQUE                       DSC bande   
4  1.104033e+09   Champs-sur-Marne    UNIQUE                            None   

  panneaux moyenn_ech revetement      highway insee_com  \
0     None         32       None      service     77108   
1     None         11       None        track     94060   
2     None         11    asphalt         path     77249   
3     None         22       None  residential     77373   
4     None         22    asphalt  residential     77083   

                     nom_voie  longueur petite_ech           nv  \
0  Rue de la Mare Longue Noue        24          3    limite 30   
1             Allée Jacquette 

### Création de variables d'étude pour les aménagements cyclables

Nous souhaitons étudier la répartition géograhique des aménagements cyclables et particulièrement leur densité en mètre par habitants. Toutefois, l'ensemble des pistes cyclables n'est pas de la même "qualité" : séparée de la route ou non, sens inverse de la circulation, type de revêtement, etc. Pour prendre en compte la qualité des aménagements cyclables, nous pouvons donner des coefficients aux mètres de pistes selon ces différents critères lors du calcul du nombre total de mètres aménagés par commune. Nous allons proposer plusieurs méthodes de calcul selon ces critères.

In [11]:
print(df_a_velo["sens_voit"].unique())
print(df_a_velo["ag"].unique())
print(df_a_velo["revetement"].unique())
print(df_a_velo["highway"].unique())
print(df_a_velo["nv"].unique())
print(df_a_velo["ad"].unique())

['DOUBLE' 'NC' 'UNIQUE']
[None 'chemin service site propre uni' 'voie verte uni' 'DSC bande'
 'autre chemin velo uni' 'piste uni' 'DSC' 'piste trottoir uni'
 'bande uni' 'cheminement trottoir uni' 'goulotte' 'chemin dedie uni'
 'chaucidou' 'cheminement uni' 'DSC piste' 'voie bus uni' 'piste bi'
 'bande bi' 'shoulder uni']
[None 'asphalt' 'compacted' 'concrete' 'unpaved' 'wood' 'paving_stones'
 'ground' 'sett' 'gravel' 'sand' 'paved' 'fine_gravel' 'dirt'
 'cobblestone' 'concrete:plates' 'concrete:lanes' 'grass'
 'cobblestone:flattened' 'metal' 'pebblestone' 'unhewn_cobblestone'
 'bricks' 'tartan' 'pavés' 'vegecol' 'artificial_turf' 'grass_paver'
 'earth' 'bitume' 'à_définir' 'rock']
['service' 'track' 'path' 'residential' 'footway' 'cycleway' 'secondary'
 'tertiary' 'living_street' 'unclassified' 'pedestrian' 'steps' 'primary'
 'primary_link' 'secondary_link' 'tertiary_link' 'trunk_link'
 'motorway_link']
['limite 30' 'hors voirie' None 'z20' 'rue pietonne' 'z30' 'escalier velo'
 'velor

In [12]:
df_a_velo['sens_voit'].isna().sum()

0

In [13]:
df_a_velo['ag'].isna().sum()

68312

In [14]:
df_a_velo['revetement'].isna().sum()

38711

In [15]:
df_a_velo['highway'].isna().sum()

0

In [16]:
df_a_velo['nv'].isna().sum()

9548

In [17]:
df_a_velo['ad'].isna().sum()

68522

Nous n'allons donc garder que le critères suivant :  type de route ("highway", cf [documentation OpenstreeMap](https://wiki.openstreetmap.org/wiki/Key:highway) ). Les autres variables ont soit trop de valeurs manquantes soit manquent d'intérêt seule (sens des voitures). Nous allons produire 2 variables : l'une pondérée avec le type de route censé refleter la qualité de la route, et une autre sans pondération, pour la longueur d'aménagements cyclables par ville.

In [18]:
highway_quality_mapping = {
    'service': 1,
    'track': 1,
    'path': 1,
    'trunk_link': 1,
    'motorway_link': 1,
    'residential': 2,
    'footway': 2,
    'cycleway': 2,
    'primary': 2,    
    'primary_link': 2,    
    'secondary': 3,
    'tertiary': 3,
    'secondary_link': 3,
    'tertiary_link': 3,
    'living_street': 3,
    'unclassified': 3,
    'pedestrian': 4,
    'steps': 4
}

In [19]:
df_a_velo['qual'] = df_a_velo['highway'].map(highway_quality_mapping)

In [20]:
quality_weights = {
    1: 1.25,
    2: 1.,
    3: 0.75,
    4: 0.5
}

In [21]:
# Nouvelle colonne pour la longueur pondérée

df_a_velo['longueur_pond'] = df_a_velo['longueur'] * df_a_velo['qual'].map(quality_weights)

In [22]:
total_longueur_pond = df_a_velo['longueur_pond'].sum()
print(total_longueur_pond) #longueur totale de l'échantillon avec majoration pondérée

13882578.0


In [23]:
total_longueur = df_a_velo['longueur'].sum()
print(total_longueur)

14174798


In [24]:
total_longueur_commune = df_a_velo.groupby(['nom_com', 'insee_com'])['longueur'].sum().reset_index()

In [25]:
total_longueur_pond_commune = df_a_velo.groupby(['nom_com', 'insee_com'])['longueur_pond'].sum().reset_index()

In [26]:
df_amenagements = pd.merge(total_longueur_commune, total_longueur_pond_commune, on=['nom_com', 'insee_com'], suffixes=('_non_pond', '_pond'))

df_amenagements['difference'] = df_amenagements['longueur_pond'] - df_amenagements['longueur']

df_amenagements.sample(10)

,nom_com,insee_com,longueur,longueur_pond,difference
997,Étampes,91223,24485,22956.25,-1528.75
580,Moisson,78410,1425,1523.75,98.75
409,Juziers,78327,9442,10257.00,815.00
977,Vémars,95641,14593,13760.00,-833.00
774,Saint-Germain-lès-Arpajon,91552,5909,6277.50,368.50
311,Fontenay-aux-Roses,92032,34752,31371.00,-3381.00
382,Hermé,77227,319,239.25,-79.75
950,Villeron,95675,608,505.00,-103.00
992,Épinay-sous-Sénart,91215,9086,10397.75,1311.75
595,Montgeroult,95422,264,259.50,-4.50


Nous avons créé nos deux indicateurs par commune : longueur et longueur pondérée par commune, mais nous avons perdu de l'information dans cette opération puisque nous perdons de ce fait les coordonnées exactes des aménagements. Cependant, pour l'étude de la densité, cela pourra aussi s'avérer utile. Surtout nous avons besoin de ces indicateurs pour le travail économétrique que nous souhaitons mener, où nous considérons alors les communes comme des individus, et la longueur des pistes cyclables comme nous variable d'intérêt. Cependant, avant de passer à la modélisation économétrique pour tenter d'expliquer le développement des aménagements dans les différentes communes, nous souhaitons représenter et analyser spatialement la répartition des aménagements, leur densité (par habitant), leur densité (par kilomètre) et notamment compléter ces analyses grâce à la disponibilité des données du Vélib.

## Les vélibs en Ile de France

In [27]:
url2 = "https://opendata.paris.fr/explore/dataset/velib-emplacement-des-stations/download/?format=geojson&timezone=Europe/Berlin&lang=fr"

In [28]:
velib = gpd.read_file(url2)

KeyboardInterrupt: 

In [ ]:
velib.sample(n=15)

## Données et ressources nécessaires pour l'étude spatiale

In [ ]:
communes = download_vectorfile_url_all(
    crs = 4326,
    borders="COMMUNE_ARRONDISSEMENT",
    values = ["75","77","78","91", "92", "93", "94","95"],
    vectorfile_format="topojson",
    filter_by="DEPARTEMENT",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022)

Un préalable au calcul des surfaces est le choix du système de projection adéquat. Dans notre cas , il s'agit de convertir les données au système de projection Lambert 93 qui est le plus approprié.

In [ ]:
communes['surface'] = communes.to_crs(2154).area

In [ ]:
communes.sort_values('surface', ascending = False)

In [ ]:
communes.rename(columns={'INSEE_COG': 'insee_com'}, inplace=True)

## Données socio-démographiques sur l'Ile de France

## Données complémentaires : accidents de la route, transports écologiques alternatifs (réseau ferré, bus)

## Base finale avec la longueur, les indicateurs géographiques, et les variables socio-démographiques

In [ ]:
base = communes[['insee_com', 'POPULATION', 'surface', 'geometry']].merge(y, how='outer', on='insee_com')

In [ ]:
base.groupby('insee_com').sum(numeric_only = True).sort_values('longueur', ascending = False)

In [ ]:
base = gpd.GeoDataFrame(base, geometry='geometry')

# Analyse descriptive et spatiale

## Répartition spatiale des aménagements cyclables : piste cyclable et station vélib

In [ ]:
# Répartition des stations vélib

fig,ax = plt.subplots(figsize=(10, 10))
velib.plot(ax = ax, color = 'green')
com.plot(ax = ax, zorder=1, edgecolor = "black", facecolor="none")
ax.set_axis_off()

In [ ]:
# Emplacement des pistes cycables (point)

fig,ax = plt.subplots(figsize=(10, 10))
a_velo.plot(ax = ax, color = 'red', alpha = 0.4, zorder=2)
communes.plot(ax = ax, zorder=1, edgecolor = "black", facecolor="none")
ax.set_axis_off()

In [ ]:
# Répartition des pistes cyclabes en île de France

fig, ax = plt.subplots(figsize=(10, 10))
dissolved = base.dissolve(by='insee_com', aggfunc='sum').reset_index()
dissolved.plot(ax=ax, column="longueur", legend=True)
ax.set_axis_off()
legend = ax.get_legend()
plt.show()

### Statistiques descriptives sur les pistes cyclables

In [ ]:
df_amenagements.describe()

In [ ]:
sns.scatterplot(df_amenagements['longueur'])
plt.title('Distribution des pistes cyclables dans les communes')